❌ Aplicando restrições na busca ❌
=============================

## Introdução



Muitos problemas de otimização com relevância científica têm uma ou mais `restrições` que devem ser levadas em consideração na hora de resolver o problema.

Lembra do `problema da mochila` que vimos em Lógica Computacional? Era um problema de otimização onde queríamos maximizar o valor dos itens colocados na mochila enquanto observávamos a restrição do peso total dos itens (do contrário, a mochila rasgava).

Uma forma de considerar essas restrições nos problemas é aplicando uma `penalidade` na função objetivo.

Vamos pensar como seria essa penalidade no problema da mochila: a função objetivo é maximizar o valor dos itens na mochila, então é um problema de maximização. A função objetivo pode ser a soma dos itens da mochila. Se fosse só isso, teríamos

$$
f = \sum_{i, i \in \mathrm{mochila}}\mathrm{valor}(i)
$$

No entanto, apenas essa função não resolve o problema! Precisamos levar em consideração o limite de peso da mochila! Para isso, penalizamos a função objetivo levando em consideração essa restrição:

$f=\begin{cases}
0 & \textrm{se peso > limite da mochila}\\
\sum_{i,i\in\mathrm{mochila}}(\mathrm{valor}(i)) & \textrm{se peso} \leq \textrm{limite da mochila}
\end{cases}$

Agora finalmente podemos seguir em frente e resolver o problema.



## Reflexões



Se usarmos a equação de $f$ acima, qual será o valor de $f$ caso não exista uma solução para um certo problema da mochila?

Na equação de $f$ acima nós usamos o valor zero para indicar que uma restrição do problema não foi satisfeita. Você consegue pensar em outra estratégia para penalizar soluções inválidas?



## Objetivo



Encontrar uma solução para o problema da mochila usando algoritmos genéticos. Considere que existem 10 itens diferentes (com pesos e valores diferentes) disponíveis para serem escolhidos.



## Descrição do problema



No problema da mochila você tem um número $n$ de itens disponíveis, cada um com um peso e um valor associado. Sua mochila tem a capacidade de carregar um número $p$ de quilogramas, sendo que mais que isso faz com que sua mochila rasgue e todos os itens dentro dela caiam no chão e se quebrem de maneira catastrófica (indesejado). Sua tarefa é encontrar um conjunto de itens (considerando os $n$ disponíveis) que maximize o valor contido dentro da mochila, porém que tenham um peso dentro da capacidade da mesma.



## Importações



In [1]:
import random

from funcoes import computa_mochila
from funcoes import funcao_objetivo_pop_mochila
from funcoes import populacao_cb as cria_populacao_inicial
from funcoes import selecao_roleta_max as funcao_selecao
from funcoes import cruzamento_ponto_simples as funcao_cruzamento
from funcoes import mutacao_cb as funcao_mutacao

## Códigos e discussão



In [28]:
### CONSTANTES

# relacionadas à busca
TAMANHO_POP = 20
NUM_GERACOES = 100
CHANCE_CRUZAMENTO = 0.5
CHANCE_MUTACAO = 0.05

# relacionadas ao problema a ser resolvido
LIMITE_DE_PESO = 15
OBJETOS = {
    # dicionário baseado no que vocês enviaram na aula de Lógica

    "Mais seguidores no Instagram": {
        "peso": 2,
        "valor": 2500,
    },
    "Clube das Wixs nos cinemas": {
        "peso": 3,
        "valor": 1500,
    },
    "Prêmio Capes": {
        "peso": 3,
        "valor": 7000,
    },
    "SmartWatch": {
        "peso": 3,
        "valor": 150,
    },
    "Macbook": {
        "peso": 0.6,
        "valor": 2400,
    },
    "Fone JBL": {
        "peso": 3.5,
        "valor": 3000,
    },
    "Um carro e uma carteira de habilitação": {
        "peso": 16,
        "valor": 1000,
    },
    "Lições em dia": {
        "peso": 8,
        "valor": 5000,
    },
    "Um Doutorado": {
        "peso": 2,
        "valor": 1500,
    },
    "Morar na beira da praia": {
        "peso": 5,
        "valor": 3000,
    },
}
NUM_OBJETOS = len(OBJETOS)
ORDEM_DOS_NOMES = list(sorted(OBJETOS.keys()))

In [29]:
# Funções locais

def funcao_objetivo_pop(populacao):
    return funcao_objetivo_pop_mochila(
        populacao, OBJETOS, LIMITE_DE_PESO, ORDEM_DOS_NOMES
    )

In [70]:
# Busca por algoritmo genético

populacao = cria_populacao_inicial(TAMANHO_POP, NUM_OBJETOS)

# variaveis para o hall da fama
melhor_fitness_ja_visto = -float("inf")
melhor_individuo_ja_visto = [0] * NUM_OBJETOS

for n in range(NUM_GERACOES):

    # Seleção
    fitness = funcao_objetivo_pop(populacao)
    populacao = funcao_selecao(populacao, fitness)

    # Cruzamento
    pais = populacao[0::2]
    maes = populacao[1::2]

    contador = 0

    for pai, mae in zip(pais, maes):
        if random.random() <= CHANCE_CRUZAMENTO:
            filho1, filho2 = funcao_cruzamento(pai, mae)
            populacao[contador] = filho1
            populacao[contador + 1] = filho2

        contador = contador + 2

    # Mutação
    for n in range(len(populacao)):
        if random.random() <= CHANCE_MUTACAO:
            individuo = populacao[n]
            populacao[n] = funcao_mutacao(individuo)

    # melhor individuo já visto até agora (hall da fama)
    fitness = funcao_objetivo_pop(populacao)
    maior_fitness = max(fitness)
    posicao = fitness.index(maior_fitness)
    individuo = populacao[posicao].copy()
    valor, peso = computa_mochila(individuo, OBJETOS, ORDEM_DOS_NOMES)
    if maior_fitness > melhor_fitness_ja_visto and peso <= LIMITE_DE_PESO:
        melhor_fitness_ja_visto = maior_fitness
        melhor_individuo_ja_visto = individuo
        print(f"Maior valor: {valor} | Peso: {peso}")


# reportando o melhor individuo encontrado
print()
print("Você deve pegar os seguintes itens:")
for pega_ou_nao, item in zip(melhor_individuo_ja_visto, ORDEM_DOS_NOMES):
    if pega_ou_nao == 1:
        print("+", item)
print()
valor_total, peso_total = computa_mochila(
    melhor_individuo_ja_visto, OBJETOS, ORDEM_DOS_NOMES
)
print(
    f"Com isso, sua mochila terá o valor de {valor_total} dinheiros "
    f"e peso de {peso_total} unidades de massa."
)

Maior valor: 12550 | Peso: 10.1
Maior valor: 14050 | Peso: 13.1
Maior valor: 15400 | Peso: 12.1
Maior valor: 16400 | Peso: 12.6

Você deve pegar os seguintes itens:
+ Macbook
+ Mais seguidores no Instagram
+ Morar na beira da praia
+ Prêmio Capes
+ Um Doutorado

Com isso, sua mochila terá o valor de 16400 dinheiros e peso de 12.6 unidades de massa.


## Discussão



No experimento de hoje realizamos uma busca com restrições para o problema da mochila por meio de uma busca exaustiva. É um problema NP difícil de maximização, já que eu quero colocar o máximo de itens possíveis na minha mochila. Então colocamos novs funções no arquivo de funcoes.py, computa_mochila, funcao_objetivo_mochilha, funcao_objetivo_pop_mochila. 
É um algoritmo de otimização com restrições, visto que, preciso de um peso limite para colocar na minha mochila ou então ela se rasgaria. 

## Conclusão

Podemos concluir que é um algortimo eficiente, já que o problema da mochila tem muitas variáveis para serem consideradas e restrições a serem cumpridas e retornou uma valor satisfatório. No exemplo da mochila, temos que usar um peso para cada objeto e um valor para que seja avaliado a possibilidade desse item entrar na mochila, de acordo com os limites estabelecidos. Além disso, vimos que apesar de ser um experimento mais complexo, voltamos a utilizar parte do código do primeiro que vimos por aqui, o de caixas binárias!

## Playground



## <font color= blue> Quais amigos eu levaria na minha mala?

In [21]:
OBJETOS2 = {
    
    "Isa Beneti": {
        "peso": 0.3,
        "valor": 100,
    },
    "Shai": {
        "peso": 0.25,
        "valor": 80,
    },
    "Pedro Sophia": {
        "peso": 0.2,
        "valor": 110,
    },
    "Isadora": {
        "peso": 0.35,
        "valor": 120,
    },
    "Jambas": {
        "peso": 0.15,
        "valor": 70,
    },
    "Daniel": {
        "peso": 0.2,
        "valor": 90,
    },
    "Gabriel": {
        "peso": 0.3,
        "valor": 150,
    },
    "Sarah": {
        "peso": 0.25,
        "valor": 120,
    },
    "João Careca": {
        "peso": 0.2,
        "valor": 80,
    },
    "Paola": {
        "peso": 0.25,
        "valor": 130,
    }
}


NUM_OBJETOS2 = len(OBJETOS2)
ORDEM_DOS_NOMES2 = list(sorted(OBJETOS2.keys()))

In [20]:
# Busca por algoritmo genético

populacao = cria_populacao_inicial(TAMANHO_POP, NUM_OBJETOS2)

# variaveis para o hall da fama
melhor_fitness_ja_visto = -float("inf")
melhor_individuo_ja_visto = [0] * NUM_OBJETOS2

for n in range(NUM_GERACOES):

    # Seleção
    fitness = funcao_objetivo_pop(populacao)
    populacao = funcao_selecao(populacao, fitness)

    # Cruzamento
    pais = populacao[0::2]
    maes = populacao[1::2]

    contador = 0

    for pai, mae in zip(pais, maes):
        if random.random() <= CHANCE_CRUZAMENTO:
            filho1, filho2 = funcao_cruzamento(pai, mae)
            populacao[contador] = filho1
            populacao[contador + 1] = filho2

        contador = contador + 2

    # Mutação
    for n in range(len(populacao)):
        if random.random() <= CHANCE_MUTACAO:
            individuo = populacao[n]
            populacao[n] = funcao_mutacao(individuo)

    # melhor individuo já visto até agora (hall da fama)
    fitness = funcao_objetivo_pop(populacao)
    maior_fitness = max(fitness)
    posicao = fitness.index(maior_fitness)
    individuo = populacao[posicao].copy()
    valor, peso = computa_mochila(individuo, OBJETOS2, ORDEM_DOS_NOMES2)
    if maior_fitness > melhor_fitness_ja_visto and peso <= LIMITE_DE_PESO:
        melhor_fitness_ja_visto = maior_fitness
        melhor_individuo_ja_visto = individuo
        print(f"Maior valor: {valor} | Peso: {peso}")


# reportando o melhor individuo encontrado
print()
print("Você deve pegar os seguintes itens:")
for pega_ou_nao, item in zip(melhor_individuo_ja_visto, ORDEM_DOS_NOMES2):
    if pega_ou_nao == 1:
        print("+", item)
print()
valor_total, peso_total = computa_mochila(
    melhor_individuo_ja_visto, OBJETOS2, ORDEM_DOS_NOMES2
)
print(
    f"Com isso, sua mochila terá o valor de {valor_total} dinheiros "
    f"e peso de {peso_total} unidades de massa."
)

Maior valor: 580 | Peso: 1.3499999999999999

Você deve pegar os seguintes itens:
+ Gabriel
+ Isadora
+ Pedro Sophia
+ Sarah
+ Shai

Com isso, sua mochila terá o valor de 580 dinheiros e peso de 1.3499999999999999 unidades de massa.
